# Download expression data of *Escherichia coli*

Reference:  
Hwang, J.-Y. & Buskirk, A. R. A ribosome profiling study of mRNA cleavage by the endonuclease RelE. 
Nucleic Acids Res ***45***, 1 (2017). doi:[10.1093/nar/gkw944](https://doi.org/10.1093/nar/gkw944)

- The reference genome (GCF_000005845.2) was downloaded from NCBI ftp server (RefSeq or GenBank) to `data/pubdata/expression/HB17NAR/reference`
- Raw fastq files were downloaded from EBI ftp server to `data/pubdata/expression/HB17NAR/fastq`


In [1]:
import gzip
import hashlib
import pandas as pd
import shutil
import time
import urllib.request 
from Bio import SeqIO
from pyscripts.config import path2
from pyscripts.datasets import DatasetDownloader
ddownloader = DatasetDownloader()

In [2]:
refs = pd.read_csv(path2.metadata/'expression'/'HB17NAR_refs.tsv', sep='\t')

In [3]:
for ref in refs.itertuples():
    # fetch genomes
    file_target = path2.pubdata/'expression'/'HB17NAR'/'reference'/f'{ref.reference_assembly}.gbff.gz'
    ddownloader.fetch_NCBI_genome(ref.ftp_path, file_target)
    
    # convert GenBank files to FASTA files
    with gzip.open(file_target, 'rt') as gbff:
        SeqIO.convert(gbff, 'gb', path2.data/'expression'/'HB17NAR'/'reference'/f'{ref.reference_assembly}.fna', 'fasta')
    time.sleep(1)

In [4]:
runs = pd.read_csv(path2.metadata/'expression'/'HB17NAR_runs.tsv', sep='\t')

In [5]:
# fetch raw reads in fastq format
for run in runs.itertuples():
    ftp_source  = f'ftp://{run.fastq_ftp}'
    file_target = (path2.pubdata/'expression'/'HB17NAR'/'fastq'/run.tag).with_suffix('.fastq.gz')
    with urllib.request.urlopen(ftp_source) as response, open(file_target, 'wb') as outfile:
        shutil.copyfileobj(response, outfile)
    with open(file_target, 'rb') as fqfile:
        md5sum = hashlib.md5(fqfile.read()).hexdigest() 
        
    assert md5sum == run.fastq_md5
    time.sleep(1)
    